## Chapter 2 - 자연어와 단어의 분산 표현


#### 2.1 자연어 처리란?
---
1. 시소러스를 활용한 기법
2. 통계 기반 기법 (모든 데이터셋 활용)
3. 추론 기반 기법 (일부 데티어셋 사용)


#### 2.2 시소러스
---
- 시소러스란 기본적으로 유의어 사전으로 '뜻이 같은 단어(동의어)'나 '뜻이 비슷한 단어(유의어)'가 한 그룹으로 분류되어 있습니다.  

    ex) 자동차 - 원동기를 장치하여 그 동력으로 바퀴를 굴려서 철길이나 가설된 선에 의하지 아니하고 땅 위를 움직이도록 만든 차
   
	![이미지](../image/image_1.png)

1. WordNet 
	- 자연어 처리 분야에서 가장 유명한 시소러스 

2. 시소러스의 문제점
    1. 시대 변화에 대응하기 어렵다.
    2. 사람을 쓰는 비용은 크다.
    3. 단어의 미묘한 차이를 표현할 수 없다.

#### 2.3 통계 기반 기법
---
1. 말뭉치
	- 말뭉치란 간단히 말하면 대량의 텍스트 데이터입니다. 다만 맹목적으로 수집된 텍스트 데이터가 아닌 자연어 처리 연구나 어플리케이션을 염두에 두고 수집된 텍스트 데이터를 일반적으로 ‘말뭉치’라고 합니다.
	- 결국 말뭉치란 텍스트 데이터에 지나지 않습니다만, 그 안에 담긴 문장들은 사람이 쓴 글입니다. 다른 시각에서 생각해보면, 말뭉치에는 자연어에 대한 사람의 ‘지식’이 충분히 담겨 있다고 볼 수 있습니다. 
	- 문장을 쓰는 방법, 단어를 선택하는 방법, 단어의 의미 등 사람이 알고 있는 자연어에 대한 지식이 포함되어 있는 것이죠.
	-  통계 기간 기법의 목표는 이처럼 사람의 지식으로 가득한 말뭉치에서 자동으로 그리고 효율적으로 그 핵심을 추출하는 것입니다.
	</br></br>


In [1]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word
    # Corpus는 단어 ID목록, word_to_id는 단어에서 단어 ID로의 딕셔너리, id_to_word는 단어 ID에서 단어로 딕셔너리를 뜻합니다

```py
text = ‘you say goodbye and I say hello’
id_to_word
>>> {0 : ’you’, 1 : ‘say’, 2 : ‘goodbye’, 3 : ‘and’, 4 : ‘I’, 5 : ‘hello’ , 6 : ‘ . ’}
word_to_id
>>> {’you’ : 0 , ‘say’ : ‘1’ , ‘goodbye’ : ‘2’ , ‘and’ : ‘3’ , ‘I’ : ‘4’ , ‘hello’ : ‘5’ , ‘ . ’ : 6}
Corpus
>>> array([0, 1, 2, 3, 4, 1, 5, 6])
```

2. 단어의 분산 표현
	- 색의 (RGB)처럼 벡터로 표현하듯, 단어의 의미를 정확하게 파악할 수 있는 벡터 표현을 분산 표현이라 한다. 
	- RGB 같은 벡터 표현은 단 3가지의 성분의 색으로 모든 색들을 더 정확하게 명시할 수 있다는 장점이 있습니다. 색끼리의 관련성 또한 벡터 표현 쪽이 더 쉽게 판단할 수 있고, 정량화 할 수 있습니다.

		ex) (R, G, B) = (170,33,22) 뭔진 모르겠지만 아무튼 빨강 계열

		그러면 ‘단어’또한 ‘색’처럼 표현할 수 있을까요? 더 정확히는 간결하고 이치에 맞는 벡터 표현을 단어라는 영역에서도 구축할 수 있을까요? 

		이제부터 우리가 원하는 것은 ‘단어의 의미’를 정확하게 파악할 수 있는 벡터표현입니다. 이를 자연어 처리 분야에서는 단어의 분산 표현 이라고 합니다.
		
	- 그럼 어떤 것을 기준으로 분산 표현으로 만들까?
	</br></br>

3. 분포 가설
	- 단어의 의미는 주변 단어에 의해 형성된다. 단어 자체에는 의미가 없다
	- “단어의 의미는 주변 단어에 의해 형성된다”라는 것을 분포 가설이라고 하며, 단어를 벡터로 표현하는 최근 연구도 대부분 이 가설에 기초합니다.

		분포 가설이 말하고자 하는 바는 매우 간단합니다. 단어 자체에는 의미가 없고, 그 단어가 사용된 ‘맥락’이 의미를 형성한다는 것이죠
		
		![b](../image/image_2.png)

	- 이것을 이용해서 단어의 분산 표현으로 만들자. 그런데 어떤 방법으로 만들지?
	</br></br>

4. 동시발생 행렬
	- 어떤 단어에 주목했을 때, 그 주변에 어떤 단어가 몇 번이나 등장하는지를 ‘세어’ 집계하는 방법. 동시발생 행렬의 원소는 두 단어가 동시에 발생한 횟수를 나타낸다.
	- 이 방법으로 단어의 분산 표현을 만들자!

	![c](../image/image_3.png)
	![d](../image/image_4.png)
	</br></br>

5. 벡터 간 유사도
	- 벡터 사이의 유사도를 측정하는 방법으로 코사인 유사도를 이용
	</br></br>

6. 유사 단어의 랭킹 표시
	- 벡터 사이의 유사도를 바탕으로 랭킹을 표시하는 함수 제작

#### 2.4 통계 기반 기법 개선하기
---
1. 상호정보량
	- 앞에서 본 동시발생 행렬의 원소는 두 단어가 동시에 발생한 횟수를 나타냅니다. 그러나 이 ‘발생’ 횟수라는것은 사실 그리 좋은 특징이 아닙니다. 

	- 고빈도 단어(많이 출현하는 단어)로 눈을 돌려보면 그 이유를 알 수 있습니다
	
	- ‘the’와 ‘car’등 그저 세어내는 것만으로는 두 단어가 의미는 없는데 강한 관련성을 갖는다고 평가된다. 이런 문제를 해결하기 위해 점별 상호정보량(PMI)이라는 척도를 이용한다.

	![f](../image/image_5.png)
    </br></br>

2. 차원 감소
	- PPMI 행렬은 어휘 수가 10만개라면 10만개의 차원이 생겨서 전부 다를 수 없게된다.
	- 원소 대부분이 0이라서 중요하지 않기에 노이즈등등에 취약하다 
	- 따라서 차원 감소를 해준다.
    </br></br>

3. SVD에 의한 차원 감소
	- 원소 대부분이 0인 행렬 또는 벡터는 희소행렬(희소벡터)라 한다. 차원 감소의 핵심은 희소벡터에서 중요한 축을 찾아내어 더 적은 차원으로 다시 표현하는 것인데, 차원 감소의 결과로 원래의 희소 벡터는 원소 대부분이 0이 아닌 값으로 구성된 ‘밀집벡터’로 변환한다. 이 조밀한 벡터야말로 우리가 원하는 단어의 분산 표현이다.
    </br></br>
    
4. PTB 데이터셋